# Weekly Payroll Data Analysis

In [2]:
# import pandas as pd 
import pandas as pd
# import matplotlib pyplot as plt 
import matplotlib.pyplot as plt
# import seaborn as sns
import seaborn as sns

In [ ]:
# Weekly payroll dictionary
weekly_payroll = {
    "1.": {"employee": "Linus Breeze", "Rate": "27.50", "Hours": "40.25", "Dependents": "4"},
    "2.": {"employee": "Juan Santiago", "Rate": "18.75", "Hours": "56.00", "Dependents": "1"},
    "3.": {"employee": "James Webb", "Rate": "28.35", "Hours": "38.00", "Dependents": "3"},
    "4.": {"employee": "Kylie Sabol", "Rate": "21.50", "Hours": "46.50", "Dependents": "6"},
    "5.": {"employee": "Amber Ali", "Rate": "19.35", "Hours": "17.00", "Dependents": "2"},
    "6.": {"employee": "Kevin Goldstein", "Rate": "17.05", "Hours": "28.00", "Dependents": "5"}
}

# New Employee information added to the dictionary
weekly_payroll["7."] = {"employee": "Lee Chang",
                        "Rate": "17.05", "Hours": "28.00", "Dependents": "5"}
weekly_payroll["8."] = {"employee": "Albert Fritz",
                        "Rate": "28.35", "Hours": "38.75", "Dependents": "3"}

# One Employee removed from payroll
weekly_payroll.pop("6.")

# Using get function to determine that employee deleted employee is no  longer in the payroll
definition = weekly_payroll.get("6.")
if definition:
    print(definition)
else:
    print("Key doesn't exist")


# wp = weekly payroll nested dictionary 
def extract_info(wp):  
    """ Extract info from nested dictionary

    Arg: 
        Weekly payroll as nested dictionary

    Returns:
        This function retuns list of Employee, Rate, Hours, Dependents
    """


# Empty list for keys
    employee = []
    rate = []
    hours = []
    dependents = []

# For loop nested dictionary
    for i in wp:
        employee.append(wp[i]["employee"])

    for i in wp:
        rate.append(float(wp[i]["Rate"]))

    for i in wp:
        hours.append(float(wp[i]["Hours"]))

    for i in wp:
        dependents.append(float(wp[i]["Dependents"]))
    return employee, rate, hours, dependents


# Total (Gross Pay)
def total_gross_pay(rate, hours):
    tgp = 0  # initial Total Gross pay
    for i in range(len(rate)):
        tgp = tgp + (rate[i] * hours[i])
    tgp = round(tgp, 2)
    return tgp


# Individual Gross Pay with overtime (1.5 per hour for employee with 40 and more hours per week)
def ind_gross_pay(rate, hours):
    igp = []
    for i in range(len(rate)):
        if hours[i] <= 40:
            igp.append(round(rate[i] * hours[i], 2))
        else:
            igp.append(round((rate[i] * hours[i]) +
                       (.5 * rate[i] * (hours[i] - 40)), 2))

    return igp


# State Tax = .06
def state_tax(igp):
    s_tax = []
    for items in igp:
        s_tax.append(round(items * .06, 2))
    return s_tax


# Federal Tax: flat_rate = .22
# Withholding_allowance_per_dependent = 38.46
def federal_tax(igp, dep):
    federal_tax = []
    for i in range(len(igp)):
        federal_tax.append(round(.22*(igp[i]-(dep[i]*38.46)), 2))
    return federal_tax


# Net Pay with Social Security and Medicare deductions
def net_pay(igp, federal_tax, state_tax, social_security, medicare):
    net_pay = []
    for i in range(len(igp)):
        net_pay.append(round(
            igp[i]-(social_security[i] + medicare[i] + federal_tax[i] + state_tax[i]), 2))
    return net_pay



# Social Security deduction  (with maximum amount applied)
def social_security_limited(ind_gross_pay, YTD_Social_Security):
    global Social_Security_Tax
    global Social_Security_Limit
    social_security = []
    for i in range(len(ind_gross_pay)):
        if (YTD_Social_Security[i] + (Social_Security_Tax * ind_gross_pay[i])) >= Social_Security_Limit:
            social_security.append(
                round(Social_Security_Limit - YTD_Social_Security[i], 2))
        else:
            social_security.append(
                round(Social_Security_Tax * ind_gross_pay[i], 2))

    return social_security


# Medicare deduction
def medicare(igp):
    global Medicare_tax
    medicare = []
    for i in range(len(igp)):
        medicare.append(round(igp[i] * Medicare_tax, 2))
    return medicare


In [4]:
# YTD Social Security
YTD_Social_Security = [4974.00, 5540.20, 4254.00, 5553.90, 3447.60]

# New Employee YTD Social Security (added)
YTD_Social_Security.append(4825.50)
YTD_Social_Security.append(5553.90)

# Maximum Social Security
Social_Security_Limit = 5553.90

# My function
def Maindata(wp):
    Employee, Rate, Hours, Dependents = extract_info(wp)
    Ind_Gross_Pay = ind_gross_pay(Rate, Hours)
    State_Tax = state_tax(Ind_Gross_Pay)
    Federal_Tax = federal_tax(Ind_Gross_Pay, Dependents)
    Medicare = medicare(Ind_Gross_Pay)
    Social_Security_Limited = social_security_limited(Ind_Gross_Pay, YTD_Social_Security)
    Net_Pay = net_pay(Ind_Gross_Pay, Federal_Tax, State_Tax, Social_Security_Limited, Medicare)

    Dependents = [int(items) for items in Dependents]

    
    Total_Rate = sum(Rate) #Total Rate
    Total_Hours = sum(Hours) # Total Hours 
    Total_YTD_Social_Security = sum(YTD_Social_Security)  # Total YTD Social Security
    Total_Ind_Gross_Pay = round(sum(Ind_Gross_Pay),2) # Total Ind. Gross Pay
    Total_Social_Security = round(sum(Social_Security_Limited),2) # Total Social Security
    Total_Medicare = sum(Medicare)# Total Medicare 
    Total_Federal_Tax = sum(Federal_Tax) # Total Federal tax
    Total_State_Tax = sum(State_Tax) # Total State Tax 
    Total_Net_Pay = sum(Net_Pay) # Total Net Pay

    Totals = [Total_Rate, Total_Hours, Total_YTD_Social_Security, Total_Ind_Gross_Pay, Total_Social_Security, 
            Total_Medicare, Total_Federal_Tax, Total_State_Tax, Total_Net_Pay]
    
    # Converted the list into the DataFrame
    df = pd.DataFrame([Rate, Hours, YTD_Social_Security, Ind_Gross_Pay, Social_Security_Limited, Medicare, Federal_Tax, State_Tax, Net_Pay],
                  index=['Rate', 'Hours', 'YTD Social Security', 'Ind Gross Pay',
                         'Social Security', 'Medicare', 'Federal Tax', 'State Tax', 'Net Pay'],
                  columns=Employee) 
    df = df.T
    # Totals list converted to DataFrame
    dfTotals = pd.DataFrame([Totals], index= ['Totals'], columns=['Rate', 'Hours', 'YTD Social Security', 'Ind Gross Pay',
                         'Social Security', 'Medicare', 'Federal Tax', 'State Tax', 'Net Pay'])
    return df, dfTotals


In [ ]:
# Reading csv and setting it = to payroll tax data (projected social security tax and medicare tax each year)  
payroll_tax_data = pd.read_csv('payroll_tax_data.csv')
rows = payroll_tax_data.shape[0] # counts rows ignoring header

#for case 1
Social_Security_Tax = payroll_tax_data.at[0, "Social_Security_tax"] 
Medicare_tax = payroll_tax_data.at[0, "Medicare_tax"]
df2021, df2021totals = Maindata(weekly_payroll) # took info from csv file for each year and used it in maindata function 

#for case 2
Social_Security_Tax = payroll_tax_data.at[1, "Social_Security_tax"] 
Medicare_tax = payroll_tax_data.at[1, "Medicare_tax"]
df2022, df2022totals = Maindata(weekly_payroll) 

#for case 3
Social_Security_Tax = payroll_tax_data.at[2, "Social_Security_tax"] 
Medicare_tax = payroll_tax_data.at[2, "Medicare_tax"]
df2023, df2023totals = Maindata(weekly_payroll)  


payroll_tax_data.head() 


In [ ]:
# The DataFrame displays combined results of each column and its sum of  totals

dfcase1 = pd.concat([df2021, df2021totals])

dfcase2 = pd.concat([df2022, df2022totals])

dfcase3 = pd.concat([df2023, df2023totals])

dfcase1

In [ ]:
# Weekly Payroll Data for each employee is plotted in the line graph format   
df2021new = df2021.drop("YTD Social Security", axis = 'columns')
x = df2021new.plot(kind='line',rot = 15,title = 'Weekly Payroll', figsize=(11,8))
x.legend(bbox_to_anchor=(1.0,1.0))
x.plot()


In [ ]:
# Summary of dataset
df2021.info()

In [ ]:
# Dimensions of dataset(7 rows and 9 columns)  
df2021.shape

In [ ]:
# Check for missing values
df2021.isnull().sum()

In [ ]:
# astype() function converts item/s to (float, integer, object or string) all of the items below are integers.  
# Original file if a float the current display is an int 
df2021.astype(int)

In [ ]:
# All info. of a single employee
df2021.loc["Linus Breeze"]

In [ ]:
# Single value for row/columns par
df2021.iloc[5:6, 1:2]

In [ ]:
# Employee with overtime
df2021.loc[df2021["Hours"] > 40]

In [ ]:
# Employee who reached Social Security Limit for a year
df2021.loc[df2021["YTD Social Security"] >= 5553.90]

In [ ]:
# Histogram for YTD Social Security
hist = sns.histplot(df2021['YTD Social Security'],kde =True)
hist.set_title("Histogram of YTD Social Security")
hist.set_xlabel(' YTD Social Security')
hist.set_xlim([3000,6000])


In [ ]:
# Histogram for YTD Social Security is skewd to the left away from mean
df2021['YTD Social Security'].skew()

In [ ]:
# Table case 1 Year 2021 (Social Security tax 0.062 and Medicare Tax 0.0145)
dfcase1.iloc[ :, [2, 4,5]]

In [ ]:
#Table case 2 Year 2022( Social Security tax 0.080 and Medicare Tax 0.0275) assumption
dfcase2.iloc[ :, [2, 4,5]]

In [ ]:
#Table case 3 Year 2023( Social Security tax 0.105 and Medicare Tax 0.0225) assumption
dfcase3.iloc[ :, [2, 4,5]]

In [ ]:
# Method 1 
# Summary statistics about data
# The describe() function gives us (numerical data) for count, mean, standard deviation(std), minimum, Q1(25%), 
# median(50%), Q3(75%), IQR(Q3 - Q1) and maximum values. 
df2021.describe()

In [ ]:
# METHOD 2 (Extracting Stastistical Data Manually)
# CENTRALITY MEASURES (mean, median, & mode)
df2021.mean() # Mean (average value that is equal to the ration of sum of values in a data set)
df2021.median() # Median (the central value of given set of values when arranged in an order) 
df2021.mode() # Mode (the most repetitive value of a given set of values has a high frequency or appears more frequently)
              # Nan (illustrates that computer cannot represent a value for mode)              

In [ ]:
# DISPERSION MEASURES (minimum and maximum values, range, variance, standard-deviation, Q1, Q2, Q3, & IQR)
df2021.min() # Minimum
df2021.max() # Maximum
df2021.max() - df2021.min() # Range (Measures the difference between the minimum and maximum value in a dataset)
df2021.var() # Variance (Measure of how data points differ from the mean) 
df2021.std() # Standard Deviation (Measures the typical deviation of individual values from the mean value in a dataset)

Q1 = df2021.quantile(0.25) # 25 percentile (Q1)
Q1

Q2 = df2021.quantile(0.5) # 50 percentile (Q2) or # Median
Q2

Q3 = df2021.quantile(0.75) # 75 percentile (Q3)
Q3

IQR = Q3 - Q1 # Interquartile Range(IQR) (The difference between the first quartile (Q1) and the third quartile (Q3) of the dataset)
IQR



In [ ]:
# Box-Whisker plot   # plt.bar(x,y)
font = {'family' : 'normal', 
        'color' : 'k',
        'weight' : 'bold',
        'size'   : 22}       

font2 = {'family' : 'normal', 
        'color' : 'k',
        'weight' : 'bold',
        'size'   : 30}      

plt = df2021.boxplot(rot=70, color= 'c', figsize=(8,4))
plt.set_title('Box-Whisker Plot', fontdict= font2)
plt.set_xlabel('Info', fontdict= font)
plt.set_ylabel('Quantity', fontdict= font)
# plt.spines['left'].set_color('black')
plt.grid(axis='x')
